# Problem 1

# Problem 2

In [20]:
# load in fashion MNIST dataset

import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import DataLoader, random_split

# download data
train = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

# split data into training/test
# split training data into training/validation
# set up data loaders
train_split = int(0.8 * len(train))
val_split = len(train) - train_split
train, val = random_split(train, [train_split, val_split])
train_loader = DataLoader(train, batch_size=64, shuffle=True)
test_loader = DataLoader(test, batch_size=64, shuffle=True)

In [1]:
# build autoencoder
# generate 10 images per class (dress, coat, sandal, sneaker)
# generate manifold plot (tips in homework)